# Load stuff up

In [1]:
import pandas as pd
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [2]:
from utils import *

In [3]:
data_dir = "../data/"

In [4]:
# HAP query mutants, expression lfc to HAP1 WT
hap1_expression_lfc = pd.read_csv(data_dir + "hap1_expression_lfc.csv", index_col=0)

# Raw qGI scores
hap1_crispr = pd.read_csv(data_dir + "hap1_crispr.csv", index_col=0)

# Expression lfc to DepMap median, then z-score transformed
depmap_expression_lfc_zscore = pd.read_csv(data_dir + "depmap_expression_lfc_zscore.csv", index_col=0)

# DepMap CRISPR gene effects, z-score transformed
depmap_crispr_zscore = pd.read_csv(data_dir + "depmap_crispr_zscore.csv", index_col=0)

hap1_expression_lfc.shape, hap1_crispr.shape, depmap_expression_lfc_zscore.shape, depmap_crispr_zscore.shape

((60, 16372), (60, 16432), (1021, 16372), (1021, 16432))

In [5]:
display(hap1_expression_lfc.head(), hap1_crispr.head(), depmap_expression_lfc_zscore.head(), depmap_crispr_zscore.head())

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,AAAS,AACS,AADACL3,AADACL4,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
ARID1A_019_min,-0.084393,0.0,-0.147231,0.157590,0.055844,-0.573164,-0.056520,-0.111263,0.0,0.0,...,0.113578,0.151927,0.148057,-0.081527,-0.237012,1.067202,0.089019,0.209173,0.222016,0.184415
TUBB_312_rich,-0.052505,0.0,0.702654,0.000000,0.000000,-0.036314,-0.081951,-0.166712,0.0,0.0,...,0.140634,0.515692,-0.544428,-0.580111,0.161650,0.126039,-0.151425,0.299858,0.247068,0.602173
RHOA_178_min,0.064315,0.0,-0.147231,0.172966,0.000000,1.067335,-0.011180,-0.202589,0.0,0.0,...,0.245787,0.249655,0.079060,-0.379479,0.036219,-0.572442,-0.069545,-0.251144,0.448150,0.035675
POLR2A_281_rich,0.120074,0.0,-0.073602,0.371187,0.000000,0.367672,-0.065692,-0.936705,0.0,0.0,...,0.252126,-0.201740,0.121699,-0.356995,0.089095,1.727093,-0.550399,-0.521173,-0.125355,0.168875
PSMC5_315_rich,0.139203,0.0,0.077268,0.000000,0.000000,-0.382643,-0.121161,-0.059450,0.0,0.0,...,-0.089155,0.388849,0.089179,0.337710,0.067586,0.318417,-0.167314,-0.185748,-0.135044,-0.150268


,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
ARID1A_019_min,0.071418,0.316690,0.010574,-0.085196,-0.023159,-0.113766,0.092965,0.049046,0.237073,-0.162579,...,0.043132,0.021004,-0.040488,0.166691,-0.142278,-0.019439,-0.082685,0.326004,-0.313797,0.205942
TUBB_312_rich,-0.154085,0.310388,0.024848,0.054087,0.014244,-0.032237,-0.094366,0.130520,-0.020056,-0.135761,...,-0.076941,-0.022916,0.358132,0.017626,-0.262726,-0.137035,0.037090,0.487719,0.213012,-0.250219
RHOA_178_min,0.202025,-0.037464,0.155645,0.111250,0.100754,-0.006131,-0.091993,-0.013058,0.029283,0.004613,...,0.174488,0.058054,-0.127421,-0.172388,0.060433,-0.035178,0.036162,0.133385,-0.110867,0.000702
POLR2A_281_rich,-0.082532,-0.269977,-0.014917,-0.162425,-0.061699,0.004124,-0.106612,-0.268794,-0.054147,0.019849,...,-0.114852,0.087124,0.168949,-0.507124,-0.233413,-0.224767,0.181330,-0.008551,0.090171,-0.092736
PSMC5_315_rich,0.105695,0.044078,-0.015011,0.002232,-0.058973,-0.005606,-0.187995,0.208284,-0.016312,0.009155,...,0.006981,0.085635,-0.039827,-0.161266,0.152414,-0.044009,0.313931,-0.259948,-0.122938,0.368591


,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,AAAS,AACS,AADACL3,AADACL4,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
ACH-000001,0.278897,-0.220332,-0.353268,-0.268837,-0.309877,-0.743399,0.519830,0.175776,-0.085295,-0.132347,...,0.930845,0.399901,-0.110074,-0.325771,1.922237,2.162248,0.364674,0.650601,-0.309991,0.459539
ACH-000004,0.813461,-0.294208,-0.359683,-0.023020,0.977080,-1.310591,1.620091,-1.042164,-0.085295,-0.132347,...,-0.134296,-0.177471,-0.474133,-0.794000,0.276593,-0.922391,-0.864624,0.786428,0.266955,1.326403
ACH-000005,0.725374,-0.275466,-0.399373,-0.370484,-0.309877,-1.352339,1.620091,-0.308089,-0.085295,-0.132347,...,1.656180,0.509160,-0.402540,-0.551458,0.495690,-0.922391,0.307256,0.654327,0.373113,2.444047
ACH-000007,-1.219399,1.548572,-0.366154,-0.385577,-0.309877,-1.360934,0.054641,0.385954,-0.085295,-0.132347,...,-0.708388,-0.150351,-0.884080,-0.571866,0.663667,-0.922391,-0.933668,0.284022,0.570775,-1.035774
ACH-000009,-0.551792,4.387945,-0.340602,-0.385577,0.469807,1.366802,0.473854,0.665283,-0.029190,-0.132347,...,0.507797,1.638779,-0.388467,0.094491,0.511794,0.344079,0.620275,1.424754,0.265194,-1.229187


,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
ACH-000001,-0.825790,1.153641,-0.120724,-2.138049,0.749360,2.491614,-0.808823,0.278632,1.873050,-0.978996,...,-1.359505,0.529369,1.837358,1.468376,2.201187,-2.375966,1.274226,0.793520,2.415169,-0.791616
ACH-000004,0.564651,0.039711,-1.116518,-1.067062,0.710438,-0.589461,2.453345,0.314213,2.007243,0.418096,...,1.702279,0.304934,0.409581,1.605519,1.531281,0.988895,-1.431617,1.104457,3.459211,1.493639
ACH-000005,-0.675136,0.654328,1.631419,0.459201,-0.386119,-1.227648,1.333600,0.098420,-0.592599,-0.686365,...,0.512223,-0.458845,0.407150,-0.712505,0.578254,-1.453687,0.371510,-0.239804,1.453696,0.900438
ACH-000007,0.068759,-0.511652,0.232759,0.373540,1.065789,0.555382,-0.236842,-0.392243,-1.249900,0.813795,...,-0.124342,-0.468698,0.226961,-0.382168,1.056793,-0.401302,-1.834813,0.189337,-1.291572,-0.904509
ACH-000009,0.458528,-0.600294,0.409748,0.175980,1.525947,-0.326908,0.155399,-0.078622,0.569985,-0.080630,...,0.606654,-0.190064,0.979833,-0.488176,-0.103982,-0.530175,-2.898226,0.966610,1.245292,0.094239


In [6]:
from vae import VAE
from torch import torch

vae = VAE(hap1_expression_lfc.shape[1])
vae.load_state_dict(torch.load("../models/vae_300_epochs.pt", map_location=torch.device('cpu')))
vae.eval()
vae

VAE(
  (fc1): Linear(in_features=16372, out_features=4096, bias=True)
  (fc1_bn): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2_mu): Linear(in_features=4096, out_features=128, bias=True)
  (fc2_mu_bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2_logvar): Linear(in_features=4096, out_features=128, bias=True)
  (fc2_logbar_bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=4096, bias=True)
  (fc3_bn): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=4096, out_features=16372, bias=True)
  (fc4_bn): BatchNorm1d(16372, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [7]:
from mlp import MLP

ARM_mlp = MLP(hap1_expression_lfc.shape[1], hap1_crispr.shape[1])
ARM_mlp.load_state_dict(torch.load("../models/depmap_MLP_ARM.pt", map_location=torch.device("cpu")))
ARM_mlp.eval()

MAML_mlp = MLP(hap1_expression_lfc.shape[1], hap1_crispr.shape[1])
MAML_mlp.load_state_dict(torch.load("../models/depmap_MLP_MAML.pt", map_location=torch.device("cpu")))
MAML_mlp.eval()

MLP(
  (fc1): Linear(in_features=16372, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=16432, bias=True)
)

In [8]:
def encode(df, pytorch_model):
    with torch.no_grad():
        return pd.DataFrame(pytorch_model.encode(torch.tensor(df.values).float()).numpy(), index=df.index) 

In [9]:
def autoencode(df, pytorch_model):
    with torch.no_grad():
        return pd.DataFrame(pytorch_model.encode(torch.tensor(df.values).float())[0].numpy(), index=df.index) 

In [10]:
data = hap1_expression_lfc
labels = hap1_crispr
data2 = depmap_expression_lfc_zscore
labels2 = depmap_crispr_zscore

In [11]:
alphas = [1e0, 1e1, 1e2, 1e3, 1e4, 1e5]

In [12]:
X_trains, y_trains, X_tests, y_tests = [], [], [], []

kfold = KFold(10, shuffle=True, random_state=42)
for (train_index, test_index) in tqdm(kfold.split(data), total=10):

    # pluck out data
    X_train = data.iloc[train_index]
    y_train = labels.iloc[train_index]
    X_test = data.iloc[test_index]
    y_test = labels.iloc[test_index]

    # fit scaler
    x_scaler = StandardScaler()
    x_scaler.fit(X_train)
    y_scaler = StandardScaler()
    y_scaler.fit(y_train)

    # scale features
    X_train = pd.DataFrame(x_scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(x_scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    y_train = pd.DataFrame(y_scaler.transform(y_train), index=y_train.index, columns=y_train.columns)
    y_test = pd.DataFrame(y_scaler.transform(y_test), index=y_test.index, columns=y_test.columns)
    
    X_trains.append(X_train)
    y_trains.append(y_train)
    X_tests.append(X_test)
    y_tests.append(y_test)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.09it/s]


In [13]:
label_df = pd.concat(y_tests)
label_df.to_csv("../output/kfold_zscore_labels.csv")

# train on HAP1, evaluate on HAP1

In [15]:
from tqdm import tqdm

prediction_df = None
best_mean_corr = -1
best_alpha = -1

for alpha in tqdm(alphas):
    pred_list = []
    
    for X_train, y_train, X_test, y_test in zip(X_trains, y_trains, X_tests, y_tests):

        model = Ridge(alpha)

        model.fit(X_train, y_train)
        y_pred_curr = model.predict(X_test)

        p_df = pd.DataFrame(y_pred_curr, index=y_test.index, columns=y_test.columns)

        pred_list.append(p_df)

    curr_prediction_df = pd.concat(pred_list)
    
    mean_corr = np.mean(evaluate_Challenge_B(curr_prediction_df, label_df))
    
    if mean_corr > best_mean_corr:
        best_mean_corr = mean_corr
        prediction_df = curr_prediction_df
        best_alpha = alpha

print(best_alpha)

100%|█████████████████████████████████████████████████████████████████| 6/6 [02:21<00:00, 23.55s/it]

1.0


In [16]:
prediction_df.to_csv("../output/direct_linear.csv")

# apply autoencoder, train on HAP1, evaluate on HAP1 

In [17]:
prediction_df = None
best_mean_corr = -1
best_alpha = -1

for alpha in tqdm(alphas):
    pred_list = []
    
    for X_train, y_train, X_test, y_test in zip(X_trains, y_trains, X_tests, y_tests):
        
        # encode features
        X_train_encoded = autoencode(X_train, vae)
        X_test_encoded = autoencode(X_test, vae)
        
        X_train = X_train_encoded
        X_test = X_test_encoded
        
        X_train.columns = X_train.columns.astype(str)
        X_test.columns = X_test.columns.astype(str)

        model = Ridge(alpha)

        model.fit(X_train, y_train)
        y_pred_curr = model.predict(X_test)

        p_df = pd.DataFrame(y_pred_curr, index=y_test.index, columns=y_test.columns)

        pred_list.append(p_df)

    curr_prediction_df = pd.concat(pred_list)
    
    mean_corr = np.mean(evaluate_Challenge_B(curr_prediction_df, label_df))
    
    if mean_corr > best_mean_corr:
        best_mean_corr = mean_corr
        prediction_df = curr_prediction_df
        best_alpha = alpha

print(best_alpha)

100%|█████████████████████████████████████████████████████████████████| 6/6 [00:22<00:00,  3.68s/it]

100.0


In [18]:
prediction_df.to_csv("../output/direct_linear_autoencoded.csv")

# train on both, evaluate on HAP1

In [19]:
prediction_df = None
best_mean_corr = -1
best_alpha = -1

for alpha in tqdm(alphas):
    pred_list = []
    
    for X_train, y_train, X_test, y_test in zip(X_trains, y_trains, X_tests, y_tests):
        # concat
        X_train = pd.concat([X_train, data2], axis=0)
        y_train = pd.concat([y_train, labels2], axis=0)

        model = Ridge(alpha)

        model.fit(X_train, y_train)
        y_pred_curr = model.predict(X_test)

        p_df = pd.DataFrame(y_pred_curr, index=y_test.index, columns=y_test.columns)

        pred_list.append(p_df)

    curr_prediction_df = pd.concat(pred_list)
    
    mean_corr = np.mean(evaluate_Challenge_B(curr_prediction_df, label_df))
    
    if mean_corr > best_mean_corr:
        best_mean_corr = mean_corr
        prediction_df = curr_prediction_df
        best_alpha = alpha

print(best_alpha)

100%|█████████████████████████████████████████████████████████████████| 6/6 [05:13<00:00, 52.17s/it]

10000.0


In [20]:
prediction_df.to_csv("../output/combined_linear.csv")

# Apply DepMap ARM derived linear representation, train on HAP1, evaluate on HAP1

In [21]:
prediction_df = None
best_mean_corr = -1
best_alpha = -1

for alpha in tqdm(alphas):
    pred_list = []
    
    for X_train, y_train, X_test, y_test in zip(X_trains, y_trains, X_tests, y_tests):
        
        # encode features
        X_train_encoded = encode(X_train, ARM_mlp)
        X_test_encoded = encode(X_test, ARM_mlp)
        
        X_train = X_train_encoded
        X_test = X_test_encoded
        
        X_train.columns = X_train.columns.astype(str)
        X_test.columns = X_test.columns.astype(str)

        model = Ridge(alpha)

        model.fit(X_train, y_train)
        y_pred_curr = model.predict(X_test)

        p_df = pd.DataFrame(y_pred_curr, index=y_test.index, columns=y_test.columns)

        pred_list.append(p_df)

    curr_prediction_df = pd.concat(pred_list)
    
    mean_corr = np.mean(evaluate_Challenge_B(curr_prediction_df, label_df))
    
    if mean_corr > best_mean_corr:
        best_mean_corr = mean_corr
        prediction_df = curr_prediction_df
        best_alpha = alpha

print(best_alpha)

100%|█████████████████████████████████████████████████████████████████| 6/6 [00:20<00:00,  3.36s/it]

1.0


In [22]:
prediction_df.to_csv("../output/DepMap_ARM_linear.csv")

# Apply DepMap MAML derived linear representation, train on HAP1, evaluate on HAP1

In [23]:
prediction_df = None
best_mean_corr = -1
best_alpha = -1

for alpha in tqdm(alphas):
    pred_list = []
    
    for X_train, y_train, X_test, y_test in zip(X_trains, y_trains, X_tests, y_tests):
        
        # encode features
        X_train_encoded = encode(X_train, MAML_mlp)
        X_test_encoded = encode(X_test, MAML_mlp)
        
        X_train = X_train_encoded
        X_test = X_test_encoded
        
        X_train.columns = X_train.columns.astype(str)
        X_test.columns = X_test.columns.astype(str)

        model = Ridge(alpha)

        model.fit(X_train, y_train)
        y_pred_curr = model.predict(X_test)

        p_df = pd.DataFrame(y_pred_curr, index=y_test.index, columns=y_test.columns)

        pred_list.append(p_df)

    curr_prediction_df = pd.concat(pred_list)
    
    mean_corr = np.mean(evaluate_Challenge_B(curr_prediction_df.T, label_df.T))
    
    if mean_corr > best_mean_corr:
        best_mean_corr = mean_corr
        prediction_df = curr_prediction_df
        best_alpha = alpha

print(best_alpha)

100%|█████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.85s/it]

10000.0


In [24]:
prediction_df.to_csv("../output/DepMap_MAML_linear.csv")